In [ ]:
import glob
import hashlib
import requests
import ConfigParser
from TwitterAPI import TwitterAPI
import sys
import matplotlib.pyplot as plt
import numpy as np
import os
import re
import collections
import time
from sklearn.cross_validation import KFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import tarfile
from urllib import urlretrieve
import pickle
%matplotlib inline


In [ ]:
def get_twitter(config_file):
    """Function to parse the Config file to get the Twitter OAuth Keys
    """
    config = ConfigParser.ConfigParser()
    config.read(config_file)
    twitter = TwitterAPI(
                   config.get('twitter', 'consumer_key'),
                   config.get('twitter', 'consumer_secret'),
                   config.get('twitter', 'access_token'),
                   config.get('twitter', 'access_token_secret'))
    return twitter

twitter = get_twitter('twitter.cfg')

In [ ]:
def robust_request(twitter, resource, params, max_tries=5):
    """ If a Twitter request fails, sleep for 15 minutes.
    Do this at most max_tries times before quitting.
    Args:
      twitter .... A TwitterAPI object.
      resource ... A resource string to request.
      params ..... A parameter dictionary for the request.
      max_tries .. The maximum number of tries to attempt.
    Returns:
      A TwitterResponse object, or None if failed.
    """
    for i in range(max_tries):
        request = twitter.request(resource, params)
        if request.status_code == 200:
            return request
        else:
            print >> sys.stderr, 'Got error:', request.text, '\nsleeping for 15 minutes.'
            sys.stderr.flush()
            time.sleep(61 * 15)

In [ ]:
def sample_tweets(twitter, limit):
    """Data Collection function , this fucntion uses the Twitter API's 'statuses/user_timeline' field,
    to collect the Public tweets for given handles
    Params: 
    1.Twitter API
    """
    tweets = []
    data=[]
    n_loops=16;
    for key,value in read_handles():
        print key
        for i_loop in range(1, n_loops):
            tweets = robust_request(twitter,'statuses/user_timeline', {'screen_name':key,'count':200,'page':i_loop})
            #pickle.dump(key+".pkl",tweets)
            data.append(tweets)
    return data

twitter_data_collected = sample_tweets(twitter, 5000)

In [ ]:
import pickle
def dump(filename,tweets):
    """Dumping the Tweets as .pkl for future usage
    Prams:
    Filename: The name of the pkl file in which tweets are to be saved
    """
    for t in tweets:
        print t['text']
    pickle.dump(tweets, open(filename, 'wb'))

In [ ]:
"""
Inventory mechanism to store the data as .txt for easy manual labelling
"""
import io
from collections import Counter
globallist = io.open('list.txt',"w")
for t in twitter_data_collected:
    
    c = []
    for t1 in t:
        if not(os.path.isdir(os.path.join(os.path.curdir+os.sep+'test'+os.sep+t1['user']['screen_name']))):
            os.makedirs(os.path.join(os.path.curdir+os.sep+'test'+os.sep+t1['user']['screen_name'])) 
        file = io.open(os.path.join(os.path.curdir+os.sep+'test'+os.sep+t1['user']['screen_name']+os.sep+t1['id_str']+'.txt'),"w",encoding='utf8')
        file.write(t1['text'])
        globallist.write(os.path.curdir+os.sep+t1['user']['screen_name']+os.sep+t1['id_str']+'\n')
        file.close()
globallist.close()
        